<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)


(array([0, 1]), array([619, 619]))

### Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [11]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection v2")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/11/19 11:41:45 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection v2' does not exist. Creating a new experiment.
2025/11/19 11:41:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/19 11:41:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/19 11:41:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/341219473519513425/runs/76edfc5baa22469ab213122a799d1b76
🧪 View experiment at: http://localhost:5000/#/experiments/341219473519513425


2025/11/19 11:41:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/19 11:41:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/341219473519513425/runs/10ba9d0a2e654201976ad57a3df32bb1
🧪 View experiment at: http://localhost:5000/#/experiments/341219473519513425


2025/11/19 11:41:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/19 11:41:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/341219473519513425/runs/f2e000650b7b494c8b33a82b4a833747
🧪 View experiment at: http://localhost:5000/#/experiments/341219473519513425


2025/11/19 11:41:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/341219473519513425/runs/18e622c19b2b4962bb5ad0d405e231df
🧪 View experiment at: http://localhost:5000/#/experiments/341219473519513425


### Register the Model

In [13]:
model_name = "XGB-Smote"
run_id = "18e622c19b2b4962bb5ad0d405e231df"  # your run ID

# correct model artifact path
model_uri = f"runs:/{run_id}/model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/11/19 11:53:24 WARNING mlflow.tracking._model_registry.fluent: Run with id 18e622c19b2b4962bb5ad0d405e231df has no artifacts at artifact path 'model', registering model based on models:/m-a2bfb5956dda4c65ae9a7dd8759d4941 instead
2025/11/19 11:53:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/341219473519513425/runs/18e622c19b2b4962bb5ad0d405e231df
🧪 View experiment at: http://localhost:5000/#/experiments/341219473519513425


Created version '1' of model 'XGB-Smote'.


In [15]:
model_name = "XGB-Smote"
run_id = "18e622c19b2b4962bb5ad0d405e231df"  # your run ID

# correct model artifact path
model_uri = f"runs:/{run_id}/model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)


Successfully registered model 'XGB-Smote'.
2025/11/19 11:57:10 WARNING mlflow.tracking._model_registry.fluent: Run with id 18e622c19b2b4962bb5ad0d405e231df has no artifacts at artifact path 'model', registering model based on models:/m-a2bfb5956dda4c65ae9a7dd8759d4941 instead
2025/11/19 11:57:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/341219473519513425/runs/18e622c19b2b4962bb5ad0d405e231df
🧪 View experiment at: http://localhost:5000/#/experiments/341219473519513425


Created version '1' of model 'XGB-Smote'.


### Load the Model

In [21]:
from mlflow import MlflowClient

client = MlflowClient()

# Assign alias "challenger" to version 1 of model XGB-Smote
client.set_registered_model_alias(
    name="XGB-Smote",
    alias="challenger",
    version="1"
)


In [24]:
from mlflow import MlflowClient

client = MlflowClient()

model = client.get_registered_model("XGB-Smote")
print("ALIASES:", model.aliases)


ALIASES: {'challenger': '1'}


In [26]:
model_name = "XGB-Smote"
model_uri = f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)


### Transition the Model to Production

In [27]:
from mlflow import MlflowClient
client = MlflowClient()

try:
    m = client.get_registered_model("anomaly-detection-prod")
    print("Exists:", m.name)
except Exception as e:
    print("NOT FOUND")


NOT FOUND


In [29]:
current_model_uri = "models:/XGB-Smote@challenger"
production_model_name = "anomaly-detection-prod"

client.copy_model_version(
    src_model_uri=current_model_uri,
    dst_name=production_model_name
)

print("Production model updated successfully!")


Production model updated successfully!


Registered model 'anomaly-detection-prod' already exists. Creating a new version of this model...
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


In [28]:
from mlflow import MlflowClient

client = MlflowClient()
client.create_registered_model("anomaly-detection-prod")

print("Production model created.")


Production model created.


In [30]:
client.set_registered_model_alias(
    name="anomaly-detection-prod",
    alias="production",
    version="1"
)


In [32]:
prod_model = mlflow.xgboost.load_model("models:/anomaly-detection-prod@production")


Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 